### 读取图片文件
* 200张图片，每张图片为112\*92\*3，将其转化为112\*92灰度图

In [41]:
from PIL import Image
import numpy as np
import os
img_data = []
content_path = 'data/'
for file in os.listdir(content_path):
    # 拼接完整文件路径
    cmp_path = os.path.join(content_path,file)
    I = Image.open(cmp_path)
    tmp_img = np.array(I.convert('L')) / 255  # 此处将取值范围为0-255的像素值进行放缩到0-1，便于计算高斯核函数
    img_data.append(tmp_img)
data = np.array(img_data)
print(data.shape)

(200, 112, 92)


### 采用K2DPCA对图像矩阵进行特征提取

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

def rbf(x, gamma=1):
    # 计算行之间欧式距离的平方，共计算C_(150)^2次
    sq_dists = pdist(x, 'sqeuclidean')
    # 把向量格式的距离转化为矩阵格式
    mat_sql_dists = squareform(sq_dists)
    return np.exp(-gamma*mat_sql_dists)

def KPCA(data, n_dims, kernel=rbf):
    # data为m*n的行矩阵
    K = kernel(data)
    N = K.shape[0]
    one_n = np.ones((N,N))
    # 此式相当于将公式4.2带入到4.3中化简得到的结果
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
    # 下面求C特征向量
    eig_values, eig_vector = np.linalg.eig(K)
    # 获得特征值降序索引
    idx = eig_values.argsort()[::-1]
    eigval = eig_values[idx][:n_dims]
    eigvector = eig_vector[:,idx][:,:n_dims]
    # 求标准正交向量组
    eigval = eigval ** (1/2)
    vi = eigvector/eigval.reshape(-1,n_dims)
    # 求数据在新的维度上的对应值
    data_n = np.dot(K,vi)
    return data_n
    
def K2DPCA(imgs,n_dim):
    '''
    data为绘图图像三维数据
    n_dim为降维的维度
    kernel为核函数
    '''
    nd_imgs = []
    for img in imgs:
        newData = KPCA(img,n_dim)
        nd_imgs.append(newData)
    return np.array(nd_imgs)

result = K2DPCA(data,5)
result.shape

(200, 112, 5)